In [ ]:
import os
import torch
import numpy as np
from torchvision.datasets import CIFAR100
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

In [ ]:
class VisionTransformer:
    def __init__(self, embed_dim, image_resolution, vision_layers, vision_width, vision_heads, 
                 vision_patch_size):
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=vision_width, kernel_size=vision_patch_size,
                               stride=vision_patch_size, bias=False)
        scale = vision_width ** -0.5
        self.cls = nn.Parameter(scale * torch.randn(width))
        self.pos_embedding = nn.Parameter(scale * torch.randn((image_resolution // vision_patch_size) ** 2 + 1, vision_width))
        self.ln1 = nn.LayerNorm(vision_width)
        self.transformer = Transformer(vision_layers, vision_width, vision_heads)
        self.ln2 = nn.LayerNorm(vision_width)
        self.projection = nn.Parameter(scale * torch.randn(width, embed_dim))

    def forward(self, x):
        x = self.conv1(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        x = x.permute(0, 2, 1)
        x = torch.cat(self.cls.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device, x]), dim=1)
        x = x + self.pos_embedding.to(x.dtype)
        x = self.ln1(x)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)
        x = self.ln2(x)
        if self.projection:
            x = x @ self.projection
        return x

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, width, heads, attention_mask=None):
        # GPT-2 Attention block
        # Masked MultiHeadAttnetion -> LayerNorm -> FF -> LayerNorm
        self.attn = nn.MultiHeadAttention(width, head)
        self.ln1 = nn.LayerNorm(width)
        self.mlp = nn.Sequential([nn.Linear(width, width*4), nn.GeLU(),
                                  nn.Linear(width*4, width)])
        self.ln2 = nn.LayerNorm(width)
        self.attention_mask = attention_mask
    
    def attention(self, x):
        attn_mask = self.attention_mask.to(dtype=x.dtype, device=x.device) if self.attn_mask is not None else None
        return self.attn(x, x, x, need_weights=False, attn_mask=attn_mask)[0]

    def forward(self, x):
        x = self.attention(self.ln1(x))
        return self.mlp(self.ln2(x))

In [ ]:

class Transformer:
    def __init__(self, transformer_layers, transformer_width, transformer_heads, attention_mask=None):
        self.blocks = nn.Sequential([AttentionBlock(transformer_width, transformer_heads, attention_mask) 
                                     for _ in range(transformer_layers)])

    def forward(self, x):
        return self.blocks(x)

In [ ]:
class CLIP:
    def __init__(self, embed_dim, image_resolution, vision_layers, vision_width, 
                 vision_patch_size, context_length, vocab_size, transformer_width,
                 transformer_heads, transformer_layers):
        vision_heads = vision_width // 64
        self.vision_encoder = VisionTransformer(embed_dim, image_resolution, vision_layers, vision_width,
                                                vision_heads, vision_patch_size)
        #self.text_encoder = Transformer(transformer_layers, transformer_width, transformer_heads)
        self.text_encoder = nn.Sequential([AttentionBlock(transformer_width, transformer_heads, attention_mask) 
                                     for _ in range(transformer_layers)])

        self.vocab_size = vocab_size
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)
        self.pos_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))
        self.ln = nn.LayerNorm(transformer_width)

        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

    def build_attention_mask(self):
        mask = torch.empty(self.context_length, self.context_length)
        mask.fill_(float("-inf"))
        mask.triu_(1) # Returns the upper triangular part of a matrix the other elements of the result tensor out are set to 0.
        return mask
    
    def encode_text(self, x):
        x = x + self.pos_embedding(x).type(self.dtype)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = self.permute(1, 0, 2)
        x = self.ln(x).type(self.dtype)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection]

    def encode_image(self, x):
        return self.vision_encoder(x)
    
    def forward(self, text, img):
        vision_output = self.vision_encoder(img)
        text_output = self.text_encoder(text)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()

        return logits_per_image, logits_per_text

In [ ]:

# https://github.com/openai/CLIP/blob/d50d76daa670286dd6cacf3bcd80b5e4823fc8e1/clip/clip.py
MODELS = {
    "ViT-B/16": "https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt",
    "ViT-L/14": "https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt",
    "ViT-L/14@336px": "https://openaipublic.azureedge.net/clip/models/3035c92b350959924f9f00213499208652fc7ea050643e8b385c2dac08641f02/ViT-L-14-336px.pt",
}

def _convert_image_to_rgb(image):
    return image.convert("RGB")

def load_params(param_path, device):
    def _transform(n_px):
        return Compose([
            Resize(n_px, interpolation=BICUBIC),
            CenterCrop(n_px),
            _convert_image_to_rgb,
            ToTensor(),
            Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ])
    
    with open(param_path) as f:
        state_dict = torch.load(f, map_location="cpu")

    model = CLIP(embed_dim,
        image_resolution, vision_layers, vision_width, vision_patch_size,
        context_length, vocab_size, transformer_width, transformer_heads, transformer_layers)

    for key in ["input_resolution", "context_length", "vocab_size"]:
        if key in state_dict:
            del state_dict[key]
    
    model.load_state_dict(state_dict)
    model = model.eval()
    
    return model, _transform(model.visual.input_resolution)


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = load_params(MODELS['ViT-B/32'], device)

In [ ]:
# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")